## Protocol

In [7]:
import pandas as pd
import os

# ===== 입력/출력 경로 =====
INPUT_FILE  = "/home/eoil/AGENT/ICASSP2026/experiments/protocol/resnet34_score.txt"
OUTPUT_FILE = "/home/eoil/AGENT/ICASSP2026/experiments/protocol/resnet34.txt"

# ===== 파라미터 =====
THRESHOLD = 0.42
TOP_N = 4000

# ===== 데이터 로드 =====
df = pd.read_csv(INPUT_FILE)

# ===== threshold 아래만 선택 =====
df_sub = df[df["cosine_score"] < THRESHOLD].copy()

# ===== threshold와의 거리 계산 =====
df_sub["distance"] = (THRESHOLD - df_sub["cosine_score"]).abs()

# ===== 가장 근접한 TOP_N 개 선택 =====
df_top = df_sub.nsmallest(TOP_N, "distance")

# ===== enroll_id / utt_id 추출 =====
def parse_ids(row):
    utt_id = os.path.splitext(os.path.basename(row["file"]))[0]
    enroll_base = os.path.splitext(os.path.basename(row["enroll_file"]))[0]
    parts = enroll_base.split("_")
    enroll_id = "_".join(parts[:2])  # LA_0007
    return enroll_id, utt_id

df_top[["enroll_id", "utt_id"]] = df_top.apply(parse_ids, axis=1, result_type="expand")

# ===== 원하는 포맷으로 저장 =====
with open(OUTPUT_FILE, "w") as fw:
    for _, row in df_top.iterrows():
        fw.write(f"{row['enroll_id']} {row['utt_id']} bonafide nontarget\n")

print(f"✅ Done. Saved {len(df_top)} entries to {OUTPUT_FILE}")

✅ Done. Saved 4000 entries to /home/eoil/AGENT/ICASSP2026/experiments/protocol/resnet34.txt


In [3]:
import pandas as pd

# ===== 경로 =====
INPUT_FILE  = "/home/eoil/AGENT/ICASSP2026/experiments/protocol/nontarget.txt"
OUTPUT_FILE = "/home/eoil/AGENT/ICASSP2026/experiments/protocol/random.txt"

# ===== 파라미터 =====
N_SAMPLES = 4000
SEED = 42  # 재현성을 위해 고정

# ===== 데이터 로드 =====
df = pd.read_csv(INPUT_FILE, sep="\s+", header=None)

# ===== 랜덤 샘플링 =====
if len(df) < N_SAMPLES:
    print(f"⚠️ Warning: 파일에 {len(df)}개밖에 없음. 전부 저장합니다.")
    df_sampled = df
else:
    df_sampled = df.sample(n=N_SAMPLES, random_state=SEED)

# ===== 저장 =====
df_sampled.to_csv(OUTPUT_FILE, sep=" ", index=False, header=False)

print(f"✅ Done. {len(df_sampled)}개 샘플 저장 완료: {OUTPUT_FILE}")


✅ Done. 4000개 샘플 저장 완료: /home/eoil/AGENT/ICASSP2026/experiments/protocol/random.txt


## EER - scoresumv1 & v2 / OC

In [2]:
import os
import numpy as np
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# ===== 경로 =====
OC_FILE = "/home/eoil/AGENT/ICASSP2026/experiments/total_Score/aasist.txt"
SCORE_FILE = "/home/eoil/AGENT/ICASSP2026/experiments/total_Score/resnet34_score.txt"
PROTO_FILE = "/home/eoil/AGENT/LA/ASVspoof2019_LA_asv_protocols/ASVspoof2019.LA.asv.eval.gi.trl.txt"

# ===== sigmoid 함수 =====
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# ===== 1. CM 점수 로드 (utt_id 단위, sigmoid 적용) =====
oc_scores = {}
with open(OC_FILE, "r") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) < 4:
            continue
        utt_id, score = parts[0], float(parts[-1])
        oc_scores[utt_id] = sigmoid(score)
        #oc_scores[utt_id] = score

print(f"CM 점수 개수: {len(oc_scores)}")

# ===== 2. Protocol 로드 (utt_id → label 매핑) =====
proto_labels = {}
with open(PROTO_FILE, "r") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) < 4:
            continue
        _, utt_id, _, label2 = parts
        proto_labels[utt_id] = 1 if label2 == "target" else 0

print(f"프로토콜 라벨 개수: {len(proto_labels)}")

# ===== 3. Score file 기준으로 점수 + 라벨 생성 =====
labels, scores = [], []
with open(SCORE_FILE, "r") as f:
    next(f)  # header skip
    for line in f:
        utt_path, enr_path, asv_score = line.strip().split(",")
        utt_id = os.path.splitext(os.path.basename(utt_path))[0]
        asv_score = float(asv_score)

        # oc score 있으면 합산
        if utt_id in oc_scores and utt_id in proto_labels:
            final_score = asv_score + oc_scores[utt_id]
            labels.append(proto_labels[utt_id])
            scores.append(final_score)

labels = np.array(labels)
scores = np.array(scores)

print(f"총 유효 샘플 개수: {len(labels)}")

# ===== 4. EER 계산 =====
fpr, tpr, thresholds = roc_curve(labels, scores, pos_label=1)
fnr = 1 - tpr
eer_idx = np.nanargmin(np.abs(fnr - fpr))
eer = (fpr[eer_idx] + fnr[eer_idx]) / 2
eer_threshold = thresholds[eer_idx]

print(f"\n✅ EER: {eer*100:.2f}%")
print(f"✅ EER threshold: {eer_threshold:.6f}")

CM 점수 개수: 71237
프로토콜 라벨 개수: 71237
총 유효 샘플 개수: 102579

✅ EER: 0.89%
✅ EER threshold: 1.501147


## EER / threshold - Rawnet

In [13]:
import os
import numpy as np
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# ===== 경로 =====
OC_FILE = "/home/eoil/AGENT/ICASSP2026/experiments/total_Score/rawnet2_score.txt"
SCORE_FILE = "/home/eoil/AGENT/ICASSP2026/experiments/total_Score/resnet34_score.txt"
PROTO_FILE = "/home/eoil/AGENT/LA/ASVspoof2019_LA_asv_protocols/ASVspoof2019.LA.asv.eval.gi.trl.txt"

# ===== sigmoid 함수 =====
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# ===== 1. CM 점수 로드 (utt_id 단위, sigmoid 적용) =====
oc_scores = {}
with open(OC_FILE, "r") as f:
    next(f)  # header skip
    for line in f:
        utt_path, enr_path, cm_score = line.strip().split(",")
        utt_id = os.path.splitext(os.path.basename(utt_path))[0]  # LA_E_1103494
        #cm_score = float(cm_score)
        cm_score = float(cm_score)
        cm_score = sigmoid(cm_score)   # 필요하면 sigmoid
        oc_scores[utt_id] = cm_score

print(f"CM 점수 개수: {len(oc_scores)}")

# ===== 2. Protocol 로드 (utt_id → label 매핑) =====
proto_labels = {}
with open(PROTO_FILE, "r") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) < 4:
            continue
        _, utt_id, _, label2 = parts
        proto_labels[utt_id] = 1 if label2 == "target" else 0

print(f"프로토콜 라벨 개수: {len(proto_labels)}")

# ===== 3. Score file 기준으로 점수 + 라벨 생성 =====
labels, scores = [], []
with open(SCORE_FILE, "r") as f:
    next(f)  # header skip
    for line in f:
        utt_path, enr_path, asv_score = line.strip().split(",")
        utt_id = os.path.splitext(os.path.basename(utt_path))[0]  # LA_E_1103494
        asv_score = float(asv_score)

        # oc score 있으면 합산
        if utt_id in oc_scores and utt_id in proto_labels:
            final_score = asv_score + oc_scores[utt_id]
            labels.append(proto_labels[utt_id])
            scores.append(final_score)

labels = np.array(labels)
scores = np.array(scores)

print(f"총 유효 샘플 개수: {len(labels)}")

# ===== 4. EER 계산 =====
fpr, tpr, thresholds = roc_curve(labels, scores, pos_label=1)
fnr = 1 - tpr
eer_idx = np.nanargmin(np.abs(fnr - fpr))
eer = (fpr[eer_idx] + fnr[eer_idx]) / 2
eer_threshold = thresholds[eer_idx]

print(f"\n✅ EER: {eer*100:.2f}%")
print(f"✅ EER threshold: {eer_threshold:.6f}")


CM 점수 개수: 71237
프로토콜 라벨 개수: 71237
총 유효 샘플 개수: 102579

✅ EER: 1.80%
✅ EER threshold: 1.469226


## random spoof 4000

In [1]:
import pandas as pd
import random

# ===== 경로 설정 =====
PROTO_FILE = "/home/eoil/AGENT/LA/ASVspoof2019_LA_asv_protocols/ASVspoof2019.LA.asv.eval.gi.trl.txt"
OUTPUT_FILE = "/home/eoil/AGENT/ICASSP2026/experiments/protocol/spoof_4000.txt"

# ===== 1. 파일 읽기 =====
with open(PROTO_FILE, "r") as f:
    lines = [line.strip() for line in f if line.strip()]

print(f"총 라인 수: {len(lines)}")

# ===== 2. spoof 라인만 필터링 =====
spoof_lines = [line for line in lines if line.split()[-1] == "spoof"]
print(f"spoof 라인 수: {len(spoof_lines)}")

# ===== 3. 랜덤 4000개 선택 =====
random.seed(42)  # 재현성을 위해 seed 고정
if len(spoof_lines) < 4000:
    print("⚠️ spoof 라인이 4000개보다 적습니다. 전체를 저장합니다.")
    subset = spoof_lines
else:
    subset = random.sample(spoof_lines, 4000)

# ===== 4. 저장 =====
with open(OUTPUT_FILE, "w") as f:
    for line in subset:
        f.write(line + "\n")

print(f"✅ 랜덤 spoof 4000개 저장 완료 → {OUTPUT_FILE}")


총 라인 수: 102579
spoof 라인 수: 63882
✅ 랜덤 spoof 4000개 저장 완료 → /home/eoil/AGENT/ICASSP2026/experiments/protocol/spoof_4000.txt


In [ ]:
import pandas as pd
import random

# ===== 경로 설정 =====
PROTO_FILE = "/home/eoil/AGENT/LA/ASVspoof2019_LA_asv_protocols/ASVspoof2019.LA.asv.eval.gi.trl.txt"
OUTPUT_FILE = "/home/eoil/AGENT/ICASSP2026/experiments/protocol/spoof.txt"

# ===== 1. 파일 읽기 =====
with open(PROTO_FILE, "r") as f:
    lines = [line.strip() for line in f if line.strip()]

print(f"총 라인 수: {len(lines)}")

# ===== 2. spoof 라인만 필터링 =====
spoof_lines = [line for line in lines if line.split()[-1] == "target"]
print(f"spoof 라인 수: {len(spoof_lines)}")

# ===== 3. 랜덤 4000개 선택 =====
random.seed(42)  # 재현성을 위해 seed 고정
if len(spoof_lines) < 20000:
    print("⚠️ spoof 라인이 4000개보다 적습니다. 전체를 저장합니다.")
    subset = spoof_lines
else:
    subset = random.sample(spoof_lines, 4000)

# ===== 4. 저장 =====
with open(OUTPUT_FILE, "w") as f:
    for line in subset:
        f.write(line + "\n")

print(f"✅ 랜덤 spoof 4000개 저장 완료 → {OUTPUT_FILE}")


총 라인 수: 102579
spoof 라인 수: 5370
✅ 랜덤 spoof 4000개 저장 완료 → /home/eoil/AGENT/ICASSP2026/experiments/protocol/target_4000.txt


## non = spoof protocol

In [2]:
import pandas as pd
import random

# === 입력/출력 경로 ===
PROTOCOL_FILE = "/home/eoil/AGENT/LA/ASVspoof2019_LA_asv_protocols/ASVspoof2019.LA.asv.eval.gi.trl.txt"
OUTPUT_FILE   = "/home/eoil/AGENT/ICASSP2026/experiments/protocol/balanced.txt"

# === 프로토콜 불러오기 ===
df = pd.read_csv(PROTOCOL_FILE, sep=" ", header=None,
                 names=["enroll_id", "utt_id", "label1", "label2"])

print("✅ 원본 분포")
print(df["label2"].value_counts())

# === 분리 ===
df_target    = df[df["label2"] == "target"]
df_spoof     = df[df["label2"] == "spoof"]
df_nontarget = df[df["label2"] == "nontarget"]

# spoof / nontarget 중 적은 쪽에 맞추기
min_len = min(len(df_spoof), len(df_nontarget))

# 랜덤 샘플링 (재현 가능하도록 random_state 고정 가능)
df_spoof_bal     = df_spoof.sample(n=min_len, random_state=42)
df_nontarget_bal = df_nontarget.sample(n=min_len, random_state=42)

# 최종 결합 (target + spoof_bal + nontarget_bal)
df_balanced = pd.concat([df_target, df_spoof_bal, df_nontarget_bal]).reset_index(drop=True)

print("\n✅ 균형 조정 후 분포")
print(df_balanced["label2"].value_counts())

# === 저장 ===
df_balanced.to_csv(OUTPUT_FILE, sep=" ", index=False, header=False)

print(f"\n💾 저장 완료: {OUTPUT_FILE}")


✅ 원본 분포
spoof        63882
nontarget    33327
target        5370
Name: label2, dtype: int64

✅ 균형 조정 후 분포
spoof        33327
nontarget    33327
target        5370
Name: label2, dtype: int64

💾 저장 완료: /home/eoil/AGENT/ICASSP2026/experiments/protocol/balanced.txt


## Finding 실험

In [16]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from scipy.special import expit  # sigmoid 함수

# ===== 파일 경로 =====
BASE = "/home/eoil/AGENT/ICASSP2026/experiments/results/findings/noatt"

files = {
    "nont": (f"{BASE}/noatt_nont_asv.txt", f"{BASE}/noatt_nont_cm.txt", "nontarget"),
    "spf":  (f"{BASE}/noatt_spf_asv.txt",  f"{BASE}/noatt_spf_cm.txt",  "spoof"),
    "tar":  (f"{BASE}/noatt_target_asv.txt", f"{BASE}/noatt_target_cm.txt", "target"),
}

dfs = []

for key, (asv_file, cm_file, trial_type) in files.items():
    # 파일 읽기
    asv_df = pd.read_csv(asv_file)
    cm_df  = pd.read_csv(cm_file)

    # 컬럼 이름 표준화
    if "cosine_score" in asv_df.columns:
        asv_df = asv_df.rename(columns={"cosine_score": "asv_score"})
    elif "score" in asv_df.columns:
        asv_df = asv_df.rename(columns={"score": "asv_score"})

    if "cm_score" not in cm_df.columns:
        last_col = [c for c in cm_df.columns if c not in ["utt_id","enroll_id","label","trial_type"]]
        if len(last_col) == 1:
            cm_df = cm_df.rename(columns={last_col[0]: "cm_score"})
        else:
            raise ValueError(f"CM 파일 {cm_file} 에서 점수 컬럼을 찾을 수 없음")

    # ✅ CM score에 sigmoid 적용
    cm_df["cm_score"] = expit(cm_df["cm_score"])  

    # 병합
    merged = pd.merge(asv_df, cm_df, on=["utt_id","enroll_id","label","trial_type"])
    merged["fused_score"] = merged["asv_score"] + merged["cm_score"]
    dfs.append(merged)

# ===== 하나의 DataFrame으로 합치기 =====
df = pd.concat(dfs, ignore_index=True)

print("✅ 데이터 예시 (sigmoid 적용 후)")
print(df.head())


✅ 데이터 예시 (sigmoid 적용 후)
         utt_id enroll_id  asv_score     label trial_type  cm_score  \
0  LA_E_3876047   LA_0030   0.380628  bonafide  nontarget  0.991096   
1  LA_E_7304633   LA_0015   0.180547  bonafide  nontarget  0.497907   
2  LA_E_3552504   LA_0029   0.243173  bonafide  nontarget  0.989239   
3  LA_E_6332493   LA_0037   0.291038  bonafide  nontarget  0.995889   
4  LA_E_2108093   LA_0012   0.280760  bonafide  nontarget  0.991074   

   fused_score  
0     1.371724  
1     0.678454  
2     1.232412  
3     1.286927  
4     1.271834  


In [18]:
import pandas as pd
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# ===== 새 점수 파일 경로 =====
NEW_CM_FILE = "/home/eoil/AGENT/ICASSP2026/experiments/results/findings/asv/aasist.txt"
NEW_ASV_FILE = "/home/eoil/AGENT/ICASSP2026/experiments/results/findings/asv/resnet34.txt"

# ===== 새 점수 불러오기 =====
# CM 파일: 형식 → LA_E_1007872_LA_0003  5.151776
cm_new = pd.read_csv(
    NEW_CM_FILE, sep=r"\s+", header=None, names=["utt_enroll", "cm_score_new"]
)

# utt_id, enroll_id 분리
cm_new[["utt_id","enroll_id"]] = cm_new["utt_enroll"].str.split("_LA_", expand=True)
cm_new["enroll_id"] = "LA_" + cm_new["enroll_id"]
cm_new = cm_new.drop(columns=["utt_enroll"])

# ✅ sigmoid 변환
cm_new["cm_score_new"] = sigmoid(cm_new["cm_score_new"].astype(float))

# ASV 파일: 형식 → LA_E_1007872,LA_0003,0.323999
asv_new = pd.read_csv(
    NEW_ASV_FILE, sep=",", header=None, names=["utt_id","enroll_id","asv_score_new"]
)
asv_new["asv_score_new"] = asv_new["asv_score_new"].astype(float)

# ===== nontarget trial만 교체 =====
df_nont = df[df["trial_type"]=="nontarget"].copy()

# ASV 점수 교체
df_nont = df_nont.merge(asv_new, on=["utt_id","enroll_id"], how="left")
df_nont["asv_score"] = df_nont["asv_score_new"].fillna(df_nont["asv_score"])
df_nont = df_nont.drop(columns=["asv_score_new"])

# CM 점수 교체
df_nont = df_nont.merge(cm_new, on=["utt_id","enroll_id"], how="left")
df_nont["cm_score"] = df_nont["cm_score_new"].fillna(df_nont["cm_score"])
df_nont = df_nont.drop(columns=["cm_score_new"])

# ✅ 타입 강제 변환
df_nont["asv_score"] = df_nont["asv_score"].astype(float)
df_nont["cm_score"]  = df_nont["cm_score"].astype(float)

# 다시 fused_score 업데이트
df_nont["fused_score"] = df_nont["asv_score"] + df_nont["cm_score"]

# ===== target, spoof는 원래 df에서 가져옴 =====
df_others = df[df["trial_type"]!="nontarget"]
df_final = pd.concat([df_others, df_nont], ignore_index=True)

print("✅ 교체 후 데이터 예시 (nontarget)")
print(df_final[df_final["trial_type"]=="nontarget"].head())


✅ 교체 후 데이터 예시 (nontarget)
            utt_id enroll_id  asv_score     label trial_type  cm_score  \
8000  LA_E_3876047   LA_0030   0.335083  bonafide  nontarget  0.987045   
8001  LA_E_7304633   LA_0015   0.288724  bonafide  nontarget  0.557085   
8002  LA_E_3552504   LA_0029   0.325690  bonafide  nontarget  0.986187   
8003  LA_E_6332493   LA_0037   0.378909  bonafide  nontarget  0.994568   
8004  LA_E_2108093   LA_0012   0.493320  bonafide  nontarget  0.983120   

      fused_score  
8000     1.322128  
8001     0.845809  
8002     1.311877  
8003     1.373477  
8004     1.476440  


In [19]:
from sklearn.metrics import roc_curve
import numpy as np

# ===== 레이블 매핑 =====
# target → 1, nontarget & spoof → 0
df["binary_label"] = df["trial_type"].map(lambda x: 1 if x=="target" else 0)

scores = df["fused_score"].values
labels = df["binary_label"].values

# ===== EER 계산 =====
fpr, tpr, thresholds = roc_curve(labels, scores, pos_label=1)
fnr = 1 - tpr
eer_idx = np.nanargmin(np.abs(fnr - fpr))
eer = (fpr[eer_idx] + fnr[eer_idx]) / 2
eer_thr = thresholds[eer_idx]

print(f"✅ EER: {eer*100:.2f}% at threshold {eer_thr:.6f}")

# ===== a-DCF2 계산 =====
# Table 3의 파라미터
pi_spf, pi_non, pi_tar = 0.01, 0.01, 0.98
C_miss, C_fa_non, C_fa_spf = 1, 10, 10

# threshold sweep
min_c_det = float("inf")
best_thr = None

for thr in thresholds:
    # miss = target인데 score<thr
    miss = np.mean((scores < thr) & (labels == 1))
    # fa_non = nontarget인데 score>=thr
    fa_non = np.mean((scores >= thr) & (df["trial_type"]=="nontarget"))
    # fa_spf = spoof인데 score>=thr
    fa_spf = np.mean((scores >= thr) & (df["trial_type"]=="spoof"))

    c_det = (
        pi_tar * C_miss * miss
        + pi_non * C_fa_non * fa_non
        + pi_spf * C_fa_spf * fa_spf
    )
    if c_det < min_c_det:
        min_c_det = c_det
        best_thr = thr

print(f"✅ a-DCF2: {min_c_det:.4f} at threshold {best_thr:.4f}")


✅ EER: 1.05% at threshold 1.511841


✅ a-DCF2: 0.0031 at threshold 1.4707


In [7]:
import pandas as pd
import numpy as np

# ===== 파일 경로 =====
CM_FILE  = "/home/eoil/AGENT/ICASSP2026/experiments/results/findings/noatt/noatt_spf_cm.txt"
ASV_FILE = "/home/eoil/AGENT/ICASSP2026/experiments/results/findings/noatt/noatt_spf_asv.txt"

# ===== Threshold 값 =====
ASV_THR   = 0.42
CM_THR    = 0.99
FUSED_THR = 1.27

# ===== 데이터 로드 =====
# ASV 파일
asv_df = pd.read_csv(ASV_FILE).rename(columns={"cosine_score":"asv_score"})

# CM 파일 (utt_id cm_score)
cm_df = pd.read_csv(CM_FILE, header=0)

# ===== 병합 (utt_id 기준) =====
df = pd.merge(asv_df, cm_df, on="utt_id", how="inner")

# ✅ CM 점수에 sigmoid 적용
df["cm_sigmoid"] = 1 / (1 + np.exp(-df["cm_score"]))

# 합산 점수 계산 (ASV + sigmoid(CM))
df["fused_score"] = df["asv_score"] + df["cm_sigmoid"]

# ===== 판정 =====
total = len(df)
df["asv_pass"]   = df["asv_score"]   >= ASV_THR
df["cm_pass"]    = df["cm_score"]    >= CM_THR
df["fused_pass"] = df["fused_score"] >= FUSED_THR

# ===== 결과 출력 =====
print(f"총 샘플 개수: {total}")
print(f"ASV ≥ {ASV_THR} : {df['asv_pass'].sum()}개 ({df['asv_pass'].mean()*100:.2f}%)")
print(f"CM ≥ {CM_THR} : {df['cm_pass'].sum()}개 ({df['cm_pass'].mean()*100:.2f}%)")
print(f"ASV+sigmoid(CM) ≥ {FUSED_THR} : {df['fused_pass'].sum()}개 ({df['fused_pass'].mean()*100:.2f}%)")

print("\n✅ 데이터 예시")
print(df.head())

print("ASV utt_id 예시:", asv_df["utt_id"].head().tolist())
print("CM utt_id 예시:", cm_df["utt_id"].head().tolist())

common_ids = set(asv_df["utt_id"]) & set(cm_df["utt_id"])
print("공통 utt_id 개수:", len(common_ids))
print("공통 utt_id 예시:", list(common_ids)[:10])


총 샘플 개수: 4000
ASV ≥ 0.42 : 3019개 (75.48%)
CM ≥ 0.99 : 51개 (1.27%)
ASV+sigmoid(CM) ≥ 1.27 : 86개 (2.15%)

✅ 데이터 예시
         utt_id enroll_id_x  asv_score label_x trial_type_x enroll_id_y  \
0  LA_E_1816476     LA_0024   0.661435     A07        spoof     LA_0024   
1  LA_E_5255877     LA_0036   0.650600     A12        spoof     LA_0036   
2  LA_E_6761968     LA_0015   0.563418     A07        spoof     LA_0015   
3  LA_E_5588882     LA_0039   0.687808     A15        spoof     LA_0039   
4  LA_E_6648266     LA_0002   0.410574     A17        spoof     LA_0002   

  label_y trial_type_y  cm_score  cm_sigmoid  fused_score  asv_pass  cm_pass  \
0     A07        spoof  1.185609    0.765955     1.427390      True     True   
1     A12        spoof -0.101240    0.474712     1.125312      True    False   
2     A07        spoof -3.614998    0.026211     0.589629      True    False   
3     A15        spoof -3.633213    0.025751     0.713559      True    False   
4     A17        spoof -5.530950    